In [1]:
import sys
sys.path.append('../build')
import numpy as np
import libry as ry

**ry-c++-log** ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '
** INFO:ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:549(1) ** run path: '/home/basti/Dokumente/Bachelorarbeit/rai-python/tutorials'
** INFO:util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '

**ry-c++-log** graph.cpp:initParameters:1379(1) ** parsed parameters:
{python,
LGP/cameraFocus:[1, 0.5, 1],
LGP/collisions:1,
LGP/stopTime:300,
LGP/stopSol:6,
opt/maxStep:0.1,
opt/verbose:6,
opt/boundedNewton!}

** INFO:util.cpp:initCmdLine:549(1) ** run path: '/home/basti/Dokumente/Bachelorarbeit/rai-python/tutorials'

** INFO:graph.cpp:initParameters:1379(1) ** parsed parameters:
{python,
LGP/cameraFocus:[1, 0.5, 1],
LGP/collisions:1,
LGP/stopTime:300,
LGP/stopSol:6,
opt/maxStep:0.1,
opt/verbose:6,
opt/boundedNewton!}



In [ ]:
# load and display a configuration
C = ry.Config()
C.addFile('../test/boxProblem.g')
D = C.view()
C.getFrameNames()

**ry-c++-log** util.cpp:cd_file:996(-2) couldn't change to directory '../test' from '/home/basti/Dokumente/Bachelorarbeit/rai-python/tutorials'


In [ ]:
# create a standard path optimization problem with 4 phases, each of .2 seconds, and 10 steps
komo = C.komo_path(5., 10, .2, False)

In [ ]:
# create tau-DOFs (time intervals) in the optimization problem and add respective constraints
komo.addTimeOptimization()

In [ ]:
# take one object, and first 'switch' it to become freely movable without costs,
# then, starting at phase .7, make it dynamic (imposing NE equations)
obj = 'ballR'
#komo.addSwitch_magic(.0, 'base', obj)
komo.makeObjectsFree(["ballR"]);

In [ ]:
# (old) a way to constrain the object position to be 'before' the box before phase 0.2
#komo.addObjective(time=[0., .2], type=OT.ineq, feature=FS.position, frames=[obj], scaleTrans=[[0.,1.,0.]], target=[1.])

In [ ]:
# objectives for collision (permanent) and target (at phase-time 4.)
komo.addObjective(type=ry.OT.sos, feature=ry.FS.accumulatedCollisions)
komo.addObjective(time=[5.], type=ry.OT.eq, feature=ry.FS.distance, frames=['target', obj], scale=[3e1])

In [ ]:
# simple structure of this solution: bounce on the bottom of the box at times 1, 2, and 3
komo.addSwitch_dynamicTrans(1., -1., 'base', obj)
komo.addInteraction_elasticBounce(2., "boxBo", obj, .8)
komo.addInteraction_elasticBounce(3., "boxBo", obj, .8)
komo.addInteraction_elasticBounce(4., "boxBo", obj, .8)

In [ ]:
komo.optimize(True)

In [ ]:
komoView = komo.view()

In [ ]:
komo.display()

In [ ]:
# read out solution: the full frame path, the tau path (time optimization), list of interaction forces
obj_path = komo.getPathFrames([obj])
tau_path = komo.getPathTau()
forces = komo.getForceInteractions()
forces

Conversion from steps to phase and time:
```
phase=0 is the FIXED initial configuration and corresponds to step=-1
step=0 is the first configuration s.t. optimization
\tau(s) gives the time between step s-1 and step s
phase(step s) = (s+1)/stepsPerPhase
time(step s) = sum i=0:s tau(i)
```

In [ ]:
D=0
komoView=0
komo=0
C=0